In [2]:
#!pip install jira

In [4]:
#!pip install gspread oauth2client pandas

In [6]:
#!pip install df2gspread

In [8]:
#!pip install slack_sdk

### Imports

In [1]:
from jira import JIRA

In [2]:
import requests
import os
import json

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from IPython.display import HTML
from df2gspread import df2gspread as d2g

In [4]:
from datetime import datetime

In [5]:
from datetime import datetime, timedelta

In [6]:
from slack_sdk import WebClient

In [7]:
current_directory = os.getcwd()
print(current_directory)

/Users/mac/Desktop/Reports


In [8]:
#pip install import_ipynb

In [13]:
import import_ipynb

In [14]:
from Tokens import API_TOKEN

In [15]:
from Tokens import SLACK_TOKEN

### Google connection 

In [8]:
# Load Google credentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/mac/Desktop/Reports/credentials.json', scope)
gc = gspread.authorize(credentials)

In [9]:
DOC_PATH = '1S1oqNVL4RDCGMWpC4wmcX1YyTbl4EodKRLnfSX9kqvU'

In [10]:
#FUNCION PARA PASAR DATAFRAME AL GDOC
def update_spreadsheet(dataframe,document,nombre_pestaña):
    d2g.upload(dataframe, document, nombre_pestaña, credentials=credentials, row_names=False)

### Jira Connection

In [17]:
# Replace these with your actual values
JIRA_SERVER = "https://jitips.atlassian.net"
EMAIL = "sol.delvalle@jitips.com"

In [12]:
# Create a JIRA client
jira = JIRA(server=JIRA_SERVER, basic_auth=(EMAIL, API_TOKEN))

### Common methods

In [13]:
def get_today():
    now = datetime.now()
    hours = int(now.strftime("%H"))
    data = now.strftime("%d/%m/%Y ")+str(hours)+now.strftime(":%M")
    return data

In [14]:
def date_format(input_string):
    input_format = '%Y-%m-%dT%H:%M:%S.%f%z'
    output_format = '%Y-%m-%d'

    datetime_obj = datetime.strptime(input_string, input_format)

    output_string = datetime_obj.strftime(output_format)

    return output_string

In [47]:
def get_sprint_end_date(input_date):
    date_obj = datetime.strptime(input_date, '%Y-%m-%d')
    end_date = date_obj + timedelta(days=1)

    end_date = end_date.strftime('%Y-%m-%d')

    return end_date

In [16]:
def get_sprint(sprint_number):
    sprint_id = sprint_number+2
    api_url = "https://jitips.atlassian.net/rest/agile/1.0/sprint/"+str(sprint_id)
    auth = (EMAIL, API_TOKEN)
    
    sprint_data = ''

    response = requests.get(api_url, auth=auth)

    if response.status_code == 200:
        sprint_data = response.json()
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
    
    return sprint_data

In [17]:
def publish_in_slack(df_message):
    client = WebClient(token=SLACK_TOKEN)
    channel_id = "#qa-results"
    df_string = df_message.to_string(index=False, justify="left", col_space=15)

    response = client.chat_postMessage(
        channel=channel_id,
        #text=message
        text=f"```\n{df_string}\n```",
    )

    print(response["ok"])

In [18]:
def qa_ready_keys():
    
    issues_list= jira.search_issues('status IN ("QA ready") order by created DESC',startAt=0, maxResults=300)
    qa_ready = ''
    
    if(len(issues_list)>0):
        for i in issues_list:
            qa_ready += str(i)+' '
    else:
        qa_ready = 'No issues in QA Ready'

    return qa_ready

In [19]:
def list_qa_ready():
    
    qa_ready= 'QA Ready: '
    issues_script= jira.search_issues('status IN ("QA ready") order by created DESC',startAt=0, maxResults=300)
    
    df = pd.DataFrame(columns=[qa_ready])
    for i in issues_script:
        df = df.append({qa_ready: str(i),
                        
                        }, ignore_index=True) 
    return df 

### Report methods

In [20]:
def test_plan_report(sprint_number, sucess_rate,sucess_android, sucess_ios, sucess_odoo,sucess_automation_mobile,
                      sucess_automation_odoo, test_passed, test_failed):
    
    sprint = get_sprint(sprint_number)
    sprint_name = sprint.get('name')
    
    issues = jira.search_issues(jql_str='issuetype= "Test Plan" and summary ~ "Sprint '+str(sprint_number)+'"')
    test_plan_key = issues[0].key
    title = 'Results of Test Plan for '+str(sprint_name)+': '+str(test_plan_key)
    
    start_date = date_format(sprint.get('startDate'))
    end_date = date_format(sprint.get('endDate'))
    
    bugs_created_sprint = len(jira.search_issues(jql_str='issuetype= Bug AND created >= "'+start_date+'" AND created <= "'+end_date+'"'))
    bugs_solved_sprint = len(jira.search_issues(jql_str='issuetype in (Bug) AND status in (DONE) AND resolved >= "'+start_date+'" AND resolved <= "'+end_date+'"  order by issuetype DESC, created DESC'))
    total_open_bugs = len(jira.search_issues(jql_str='issuetype in (Bug) AND status not in (DONE, "QA ready", Canceled) order by issuetype DESC, created DESC'))
        
    data = {
        title: [ '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Start date: '+str(start_date), 'End date: '+str(end_date)],
        'B': [ '', 'Bugs created during '+str(sprint_name), bugs_created_sprint, '', 'We got a '+sucess_rate+' of Success Rate', '', 'Results of test cases executed', 'Passed', 'Failed', '', 'Success Rate by Environment', 'Android', 'iOS', 'Odoo', 'Automation Mobile', 'Automation Odoo', '', ''],
        'C': [ '', '', '', '', '', '', '', test_passed, test_failed, '', '', sucess_android, sucess_ios, sucess_odoo, sucess_automation_mobile, sucess_automation_odoo, '', ''],
        'D': [ '', 'Bugs solved/closed', bugs_solved_sprint, '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        'E': [ '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        'F': [ '', 'Total open bugs', total_open_bugs, '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
        'G': [ '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
    }

    df = pd.DataFrame(data=data)
    
    return df

In [21]:
#dir(jira)

In [22]:
def qa_daily_report(sprint_number):
    
    sprint = get_sprint(sprint_number)
    sprint_name = sprint.get('name')
    title = 'Daily report: '+sprint_name+' - '+get_today()

    start_date = date_format(sprint.get('startDate'))
    end_date = date_format(sprint.get('endDate'))
    
    qa_ready = jira.search_issues(jql_str='status IN ("QA ready") order by created DESC')
    dev_done = jira.search_issues(jql_str='status IN ("Dev done") order by created DESC')
        
    bugs_sprint = jira.search_issues(jql_str='issuetype= Bug AND created >= "'+start_date+'" AND created <= "'+end_date+'"')
    bugs_solved_sprint = jira.search_issues(jql_str='issuetype in (Bug) AND status in (DONE) AND resolved >= "'+start_date+'" AND resolved <= "'+end_date+'"  order by issuetype DESC, created DESC')
    total_bugs = jira.search_issues(jql_str='issuetype in (Bug) AND status not in (DONE, "QA ready", Canceled) order by issuetype DESC, created DESC')

    data = {
        title: [ '','BUGS: ','Created '+sprint_name, 'Solved/Closed', 'Total open bugs', 'Dev DONE','', 'QA READY TICKETS', qa_ready_keys(), '', 'Sprint start date: '+str(start_date), 'Sprint end date: '+str(end_date)],
        '': [ '','',len(bugs_sprint), len(bugs_solved_sprint), len(total_bugs),len(dev_done),'',  '', '', '', '', ''],
    }

    df = pd.DataFrame(data=data)
    
    # SLACK
    data_slack = {
        title: [ '','BUGS: ','Created '+sprint_name+': '+str(len(bugs_sprint)), 'Solved/Closed: '+str(len(bugs_solved_sprint)), 'Total open bugs: '+str(len(total_bugs)), 'Dev DONE: '+str(len(dev_done)),'', 'QA READY TICKETS', qa_ready_keys(), '', 'Sprint start date: '+str(start_date), 'Sprint end date: '+str(end_date)],
    }

    df_slack = pd.DataFrame(data=data_slack)
    
    
    return df, df_slack

In [23]:
#data = qa_daily_report(13)

In [24]:
#data[0]

In [25]:
#data[1]

In [26]:
def get_all_bugs():
    
    summary = 'Summary - Last update: '+get_today()
    
    issues_list= jira.search_issues('issuetype = Bug AND status not in (DONE, "QA ready", Canceled) order by issuetype DESC, created DESC')
    
    df = pd.DataFrame(columns=['Key',summary,'Assignee','Reporter','Status','Updated'])
    
    if(len(issues_list)>0):
        for i in issues_list:
            assignee = i.fields.assignee
            if(i.fields.assignee==None):
                assignee = 'Unassigned'
                
            
            df2 = pd.DataFrame([[ str(i), 
                                 str(i.fields.summary),
                                 str(assignee),
                                 str(i.fields.reporter.displayName),
                                 str(i.fields.status.name),
                                 date_format(str(i.fields.updated))]],
                           columns=['Key',summary,'Assignee','Reporter','Status','Updated'])
        
            df = pd.concat([df, df2], ignore_index=True)
    
    return df

In [27]:
#get_all_bugs()

In [28]:
def get_dev_done_issues():
    summary = 'Summary - Last update: '+get_today()
    
    issues_list= jira.search_issues('status IN ("Dev done") order by created DESC',startAt=0, maxResults=300)
    
    df = pd.DataFrame(columns=['Key','Type',summary,'Assignee','Updated' ])
    
    if(len(issues_list)>0):
        for i in issues_list:
            df2 = pd.DataFrame([[i, str(i.fields.issuetype.name),
                                 str(i.fields.summary),
                                 str(i.fields.assignee.displayName),
                                date_format(str(i.fields.updated))]],
                           columns=['Key','Type',summary,'Assignee','Updated'])
            df = pd.concat([df, df2], ignore_index=True)
    
    return df

In [29]:
def get_qa_ready_issues():
    summary = 'Summary - Last update: '+get_today()
    
    issues_list= jira.search_issues('status IN ("QA ready") order by created DESC',startAt=0, maxResults=300)
    
    df = pd.DataFrame(columns=['Key','Type',summary,'Assignee','Updated' ])
    
    if(len(issues_list)>0):
        for i in issues_list:
            df2 = pd.DataFrame([[i, str(i.fields.issuetype.name),
                                 str(i.fields.summary),
                                 str(i.fields.assignee.displayName),
                                date_format(str(i.fields.updated))]],
                           columns=['Key','Type',summary,'Assignee','Updated'])
            df = pd.concat([df, df2], ignore_index=True)

    return df

In [42]:
def detail_active_bugs():
        
    bugs_backend = jira.search_issues(jql_str='issuetype = Bug AND status not in (DONE, Canceled) AND labels=bug-backend order by issuetype DESC, created DESC')
    bugs_mobile = jira.search_issues(jql_str='issuetype = Bug AND status not in (DONE, Canceled) AND labels=Mobile order by issuetype DESC, created DESC')
    bugs_odoo = jira.search_issues(jql_str='issuetype = Bug AND status not in (DONE, Canceled) AND labels=Website order by issuetype DESC, created DESC')
    bugs_devops = jira.search_issues(jql_str='issuetype = Bug AND status not in (DONE, Canceled) AND labels=bug-devops order by issuetype DESC, created DESC')

    data_slack = {
        'ACTIVE BUGS DETAIL: ': ['Backend : '+str(len(bugs_backend)), 'Mobile : '+str(len(bugs_mobile)), 'Odoo : '+str(len(bugs_odoo)), 'Devops : '+str(len(bugs_devops))],
    }

    df_slack = pd.DataFrame(data=data_slack)
    
    
    return df_slack

In [31]:
def update_daily_report(sprint):
    data = qa_daily_report(sprint)
    spreadsheet = data[0]
    publish_in_slack(data[1])
    update_spreadsheet(spreadsheet,DOC_PATH,'DAILY_REPORT')

In [65]:
def dev_team_reports(sprint_number):
    
    sprint = get_sprint(sprint_number)
    sprint_name = sprint.get('name')
    title = 'Bugs Summary by team: '+sprint_name+' (Date: '+get_today()+')'

    start_date = date_format(sprint.get('startDate'))
    end_date = get_sprint_end_date(date_format(sprint.get('endDate')))
    
    # Mobile
    mob_reported_sprint = jira.search_issues(jql_str='issuetype= Bug AND created >= "'+start_date+'" AND created <= "'+end_date+'" AND labels=Mobile')
    mob_done_sprint = jira.search_issues(jql_str='issuetype= Bug AND status in (DONE, "QA ready") AND labels=Mobile AND created >= "'+start_date+'" AND created <= "'+end_date+'"')
    mob_carry_old_sprint = jira.search_issues(jql_str='issuetype = Bug AND status not in (DONE, "QA ready", Canceled) AND created < "'+start_date+'" AND labels=Mobile order by issuetype DESC, created DESC')
    mob_done_old_sprint = jira.search_issues(jql_str='issuetype in (Bug) AND status in (DONE, "QA ready") AND created < "'+start_date+'" AND resolved >= "'+start_date+'" AND resolved <= "'+end_date+'" AND labels=Mobile order by issuetype DESC, created DESC')
    
    # Odoo
    odoo_reported_sprint = jira.search_issues(jql_str='issuetype= Bug AND created >= "'+start_date+'" AND created <= "'+end_date+'" AND labels in (Odoo, Website)')
    odoo_done_sprint = jira.search_issues(jql_str='issuetype= Bug AND status in (DONE, "QA ready") AND labels in (Odoo, Website) AND created >= "'+start_date+'" AND created <= "'+end_date+'"')
    odoo_carry_old_sprint = jira.search_issues(jql_str='issuetype = Bug AND status not in (DONE, "QA ready", Canceled) AND created < "'+start_date+'" AND labels in (Odoo, Website) order by issuetype DESC, created DESC')
    odoo_done_old_sprint = jira.search_issues(jql_str='issuetype in (Bug) AND status in (DONE, "QA ready") AND created < "'+start_date+'" AND resolved >= "'+start_date+'" AND resolved <= "'+end_date+'" AND labels in (Odoo, Website) order by issuetype DESC, created DESC')
    
    # Backend
    be_reported_sprint = jira.search_issues(jql_str='issuetype= Bug AND created >= "'+start_date+'" AND created <= "'+end_date+'" AND labels=bug-backend')
    be_done_sprint = jira.search_issues(jql_str='issuetype= Bug AND status in (DONE, "QA ready") AND labels=bug-backend AND created >= "'+start_date+'" AND created <= "'+end_date+'"')
    be_carry_old_sprint = jira.search_issues(jql_str='issuetype = Bug AND status not in (DONE, "QA ready", Canceled) AND created < "'+start_date+'" AND labels=bug-backend order by issuetype DESC, created DESC')
    be_done_old_sprint = jira.search_issues(jql_str='issuetype in (Bug) AND status in (DONE, "QA ready") AND created < "'+start_date+'" AND resolved >= "'+start_date+'" AND resolved <= "'+end_date+'" AND labels=bug-backend order by issuetype DESC, created DESC')
    
    # Devops
    dops_reported_sprint = jira.search_issues(jql_str='issuetype= Bug AND created >= "'+start_date+'" AND created <= "'+end_date+'" AND labels=bug-devops')
    dops_done_sprint = jira.search_issues(jql_str='issuetype= Bug AND status in (DONE, "QA ready") AND labels=bug-devops AND created >= "'+start_date+'" AND created <= "'+end_date+'"')
    dops_carry_old_sprint = jira.search_issues(jql_str='issuetype = Bug AND status not in (DONE, "QA ready", Canceled) AND created < "'+start_date+'" AND labels=bug-devops order by issuetype DESC, created DESC')
    dops_done_old_sprint = jira.search_issues(jql_str='issuetype in (Bug) AND status in (DONE, "QA ready") AND created < "'+start_date+'" AND resolved >= "'+start_date+'" AND resolved <= "'+end_date+'" AND labels=bug-devops order by issuetype DESC, created DESC')
        
    data = {
        title: [ '','','', '', '', '','', '', '', '', '', '',  '', '', '', '', ''],
        'B': [ '','Dev Team','Mobile', 'Odoo', 'Backend ','Devops','',  '', '', '', '', '',  '', '', '', '', ''],
        'C': [ '','Reported '+sprint_name,len(mob_reported_sprint), len(odoo_reported_sprint),len(be_reported_sprint),len(dops_reported_sprint), '', '', '', '', '',  '', '', '', '', '', ''],
        'D': [ '','Resolved '+sprint_name,len(mob_done_sprint), len(odoo_done_sprint), len(be_done_sprint),len(dops_done_sprint), '', '', '', '', '',  '', '', '', '', '', ''],
        'E': [ '','Active from previous Sprints',len(mob_carry_old_sprint), len(odoo_carry_old_sprint), len(be_carry_old_sprint),len(dops_carry_old_sprint), '', '', '', '', '',  '', '', '', '', '', '* Bugs active on '+sprint_name+' reported on previous Sprint vs resolved'],
        'F': [ '','Resolved of previous Sprints',len(mob_done_old_sprint), len(odoo_done_old_sprint), len(be_done_old_sprint),len(dops_done_old_sprint), '', '', '', '', '',  '', '', '', '', '', ''],
        'G': [ '','','', '', '','','',  '', '', '', '', '',  '', '', '', '', '']
    }

    df = pd.DataFrame(data=data)

    return df

In [66]:
dev_teams =dev_team_reports(13)

In [67]:
dev_teams

,Bugs Summary by team: Sprint 13 (Date: 04/09/2023 15:02),B,C,D,E,F,G
0,,,,,,,
1,,Dev Team,Reported Sprint 13,Resolved Sprint 13,Active from previous Sprints,Resolved of previous Sprints,
2,,Mobile,8,6,3,20,
3,,Odoo,5,5,0,3,
4,,Backend,6,4,2,1,
5,,Devops,3,2,6,0,
6,,,,,,,
7,,,,,,,
8,,,,,,,
9,,,,,,,


In [68]:
update_spreadsheet(dev_teams,DOC_PATH,'DEV_TEAMS')

### Update reports

In [32]:
data = qa_daily_report(13)
spreadsheet = data[0]
#publish_in_slack(data[1])
update_spreadsheet(spreadsheet,DOC_PATH,'DAILY_REPORT')

In [33]:
publish_in_slack(data[1])

True


In [41]:
publish_in_slack(detail_active_bugs())

True


In [35]:
#update_daily_report(13)

In [36]:
update_spreadsheet(get_dev_done_issues(),DOC_PATH,'DEV_DONE') 

In [37]:
update_spreadsheet(get_qa_ready_issues(),DOC_PATH,'QA_READY') 

In [38]:
update_spreadsheet(get_all_bugs(),DOC_PATH,'ALL_BUGS') 

### Test Plan Report

In [67]:
sprint_number = 12
sucess_rate = '68%'
    
sucess_android = '57%'
sucess_ios = '55%'
sucess_odoo = '16%'
sucess_automation_mobile = '92%'
sucess_automation_odoo ='72%'
    
test_passed = '37'
test_failed = '17'

In [ ]:
test_plan_df = test_plan_report(sprint_number, sucess_rate,sucess_android, sucess_ios, sucess_odoo,sucess_automation_mobile,
                      sucess_automation_odoo, test_passed, test_failed)

In [ ]:
update_spreadsheet(test_plan_df,DOC_PATH,'TEST_PLAN')

# Code to improve

In [89]:
from IPython.display import HTML
import pandas as pd

df = pd.DataFrame({
    'name':['Softhints', 'DataScientyst'],
    'url':['https://www.softhints.com', 'https://datascientyst.com'],
    'url2':['https://www.blog.softhints.com/tag/pandas', 'https://datascientyst.com/tag/pandas']
})

df

,name,url,url2
0,Softhints,https://www.softhints.com,https://www.blog.softhints.com/tag/pandas
1,DataScientyst,https://datascientyst.com,https://datascientyst.com/tag/pandas


In [90]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val, val)

In [91]:
styled_table = (df.style
                .format({'url': make_clickable})
                .render()
               )
HTML(styled_table)

/tmp/ipykernel_11698/3094310033.py:3: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  .render()


,name,url,url2
0,Softhints,https://www.softhints.com,https://www.blog.softhints.com/tag/pandas
1,DataScientyst,https://datascientyst.com,https://datascientyst.com/tag/pandas


In [ ]:
from IPython.display import HTML

df = pd.DataFrame({'name':['Pandas', 'Linux']})

df['name'] = df['name'].apply(lambda x: f'<a href="http://softhints.com/tutorial/{x}">{x}</a>')
HTML(df.to_html(escape=False))